<a href="https://colab.research.google.com/github/luck058/kaggle-gold-price-analysis/blob/model-1/model_1_kaggle_gold_price_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [110]:
!git clone https://github.com/luck058/kaggle-gold-price-analysis

%cd kaggle-gold-price-analysis

Cloning into 'kaggle-gold-price-analysis'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 52 (delta 27), reused 20 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 936.95 KiB | 12.33 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/kaggle-gold-price-analysis/kaggle-gold-price-analysis


# Create X, y

In [111]:
df = pd.read_csv('financial_regression_cleaned.csv')

# Only care about sp500
df = df[['sp500 open', 'sp500 high', 'sp500 low', 'sp500 close', 'sp500 volume', 'sp500 high-low']].reset_index(drop=True)

display(df.head())
print(df.shape)

,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low
0,114.49,115.14,114.42,114.93,115646960.0,0.72
1,114.73,114.84,113.20,113.64,212252769.0,1.64
2,113.62,115.13,113.59,115.06,138671890.0,1.54
3,114.28,114.45,112.98,113.89,216330645.0,1.47
4,113.92,114.27,111.56,111.70,344747028.0,2.71


(3719, 6)


In [112]:
df["sp500 close diff"] = df["sp500 close"].diff()

display(df.head())
print(df.shape)

,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,sp500 close diff
0,114.49,115.14,114.42,114.93,115646960.0,0.72,NaN
1,114.73,114.84,113.20,113.64,212252769.0,1.64,-1.29
2,113.62,115.13,113.59,115.06,138671890.0,1.54,1.42
3,114.28,114.45,112.98,113.89,216330645.0,1.47,-1.17
4,113.92,114.27,111.56,111.70,344747028.0,2.71,-2.19


(3719, 7)


In [113]:
def create_lag(df, column, lags):
    for lag in range(1, lags):
        if np.log2(lag) % 1 == 0:
            df[f'{column}-{lag}'] = df[column].shift(lag)

In [114]:
create_lag(df, 'sp500 close diff', 20)
display(df.head())

,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,sp500 close diff,sp500 close diff-1,sp500 close diff-2,sp500 close diff-4,sp500 close diff-8,sp500 close diff-16
0,114.49,115.14,114.42,114.93,115646960.0,0.72,NaN,NaN,NaN,NaN,NaN,NaN
1,114.73,114.84,113.20,113.64,212252769.0,1.64,-1.29,NaN,NaN,NaN,NaN,NaN
2,113.62,115.13,113.59,115.06,138671890.0,1.54,1.42,-1.29,NaN,NaN,NaN,NaN
3,114.28,114.45,112.98,113.89,216330645.0,1.47,-1.17,1.42,-1.29,NaN,NaN,NaN
4,113.92,114.27,111.56,111.70,344747028.0,2.71,-2.19,-1.17,1.42,NaN,NaN,NaN


## Create y

In [115]:
y_col = "sp500 close diff"

df[y_col] = df[y_col].shift(-1)

display(df.head())

,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,sp500 close diff,sp500 close diff-1,sp500 close diff-2,sp500 close diff-4,sp500 close diff-8,sp500 close diff-16
0,114.49,115.14,114.42,114.93,115646960.0,0.72,-1.29,NaN,NaN,NaN,NaN,NaN
1,114.73,114.84,113.20,113.64,212252769.0,1.64,1.42,NaN,NaN,NaN,NaN,NaN
2,113.62,115.13,113.59,115.06,138671890.0,1.54,-1.17,-1.29,NaN,NaN,NaN,NaN
3,114.28,114.45,112.98,113.89,216330645.0,1.47,-2.19,1.42,-1.29,NaN,NaN,NaN
4,113.92,114.27,111.56,111.70,344747028.0,2.71,-2.49,-1.17,1.42,NaN,NaN,NaN


In [116]:
df = df.dropna(axis=0).reset_index(drop=True)
display(df.head())

,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,sp500 close diff,sp500 close diff-1,sp500 close diff-2,sp500 close diff-4,sp500 close diff-8,sp500 close diff-16
0,107.13,108.15,106.27,107.22,337698759.0,1.88,-0.21,-0.77,0.22,-0.55,-1.26,-1.29
1,107.05,107.60,106.11,107.01,240337447.0,1.49,1.12,1.33,-0.77,-3.39,-1.18,1.42
2,106.87,108.25,106.25,108.13,223348428.0,2.00,-0.09,-0.21,1.33,0.22,1.67,-1.17
3,106.99,108.10,106.51,108.04,304335521.0,1.59,1.70,1.12,-0.21,-0.77,1.32,-2.19
4,108.86,109.85,107.82,109.74,159025359.0,2.03,0.52,-0.09,1.12,1.33,-0.55,-2.49


In [117]:
y = df[y_col]
X = df.drop(y_col, axis=1)

print("y:")
display(pd.Series(y).head())
print("X:")
display(X.head())

print("len(y):")
print(len(y))
print("X.shape:")
print(X.shape)

y:


,sp500 close diff
0,-0.21
1,1.12
2,-0.09
3,1.70
4,0.52


X:


,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,sp500 close diff-1,sp500 close diff-2,sp500 close diff-4,sp500 close diff-8,sp500 close diff-16
0,107.13,108.15,106.27,107.22,337698759.0,1.88,-0.77,0.22,-0.55,-1.26,-1.29
1,107.05,107.60,106.11,107.01,240337447.0,1.49,1.33,-0.77,-3.39,-1.18,1.42
2,106.87,108.25,106.25,108.13,223348428.0,2.00,-0.21,1.33,0.22,1.67,-1.17
3,106.99,108.10,106.51,108.04,304335521.0,1.59,1.12,-0.21,-0.77,1.32,-2.19
4,108.86,109.85,107.82,109.74,159025359.0,2.03,-0.09,1.12,1.33,-0.55,-2.49


len(y):
3701
X.shape:
(3701, 11)


# Models

In [118]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import cross_validate


In [119]:
np.random.seed(1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [120]:
class PredictZero:
    """Model which just predicts y as 0 irrespective of X"""
    def fit(self, X, y):
        pass

    def predict(self, X):
        return np.zeros(len(X))

    def score(self, X, y):
        y_pred = self.predict(X)
        return r2_score(y, y_pred)

    def get_params(self, deep=True):
        return {}

In [121]:
class PredictMean:
    """Model which just predicts y as the mean of y in the training set irrespective of X"""
    def fit(self, X, y):
        self.mean_y = np.mean(y)

    def predict(self, X):
        return np.full(len(X), self.mean_y)

    def score(self, X, y):
        y_pred = self.predict(X)
        return r2_score(y, y_pred)

    def get_params(self, deep=True):
        return {}

In [ ]:
! pip install ISLP

In [128]:
from ISLP.bart import BART
from sklearn.ensemble import RandomForestRegressor


In [129]:
regression_models = {"predict_zero": PredictZero(),
                     "predict_mean": PredictMean(),
                     "linear_regression": LinearRegression(),
                     "bart": BART(burnin=5, ndraw=15),
                     "random_forest": RandomForestRegressor(n_estimators=100)
}

classifier_models = {"logistic_regression": LogisticRegression()}

In [130]:
def print_cv_results(results, z=1.96):
    """Prints the mean with confidence intervals (z-score can be modified, default 95% confidence)"""
    mean = np.mean(results)
    std_err = np.std(results) / np.sqrt(len(results))
    return f"{round(mean, 2)} " + u"\u00B1" + f"{round(z * std_err, 2)}"

In [131]:
models = regression_models

for model_name in models:
    cv_results = cross_validate(models[model_name], X_train, y_train, scoring=["r2", "neg_mean_squared_error"], cv=10)
    print(f"Model: {model_name}")
    print(f"R2: {print_cv_results(cv_results['test_r2'])}")
    print(f"MSE: {print_cv_results(cv_results['test_neg_mean_squared_error'] * (-1))}")
    print()

Model: predict_zero
R2: -0.0 ±0.0
MSE: 9.78 ±0.85

Model: predict_mean
R2: -0.0 ±0.0
MSE: 9.77 ±0.85

Model: linear_regression
R2: 0.0 ±0.02
MSE: 9.73 ±0.87

Model: bart
R2: -0.03 ±0.02
MSE: 10.08 ±0.93

Model: random_forest
R2: -0.07 ±0.05
MSE: 10.5 ±1.11

